In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

from dateutil.parser import parse

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
%run ../../tablecloth.io-storytent-201811/pylib/metaphone.py 


**********************************************************************
File "/Users/eric/proj/virt-proj/data-project/sdrdl-data-projects/tablecloth.io/tablecloth.io-storytent-201811/pylib/metaphone.py", line 19, in __main__
Failed example:
    dm(u'Bartoš'), dm(u'Bartosz'), dm(u'Bartosch'), dm(u'Bartos')
Expected:
    (('PRT', ''), ('PRTS', 'PRTX'), ('PRTX', ''), ('PRTS', ''))
Got:
    (('PRTS', ''), ('PRTS', 'PRTX'), ('PRTX', ''), ('PRTS', ''))
**********************************************************************
1 items had failures:
   1 of   4 in __main__
***Test Failed*** 1 failures.


In [3]:
pkg = mp.jupyter.open_package()
#pkg = mp.jupyter.open_source_package()
pkg

In [20]:
from datetime import timedelta, datetime
from dateutil.parser import parse


source = pkg.resource('source').dataframe()
source['form'] = ''
source['client_id'] = source.index
source['last_dm'] = source.last_name.apply(lambda v: dm(v)[0])
source['first_dm'] = source.first_name.apply(lambda v: dm(v)[0])
source.rename(columns={'gender':'sex', 'checkin_feeling':'feeling'},inplace=True)

def conv_bd(v):
    try:
        return parse(v).date()
    except ValueError:
        return None

def convt_date(v):
    if isinstance(v, str):
        
        try:
            return parse(v).date()
        except ValueError:
            pass
        
        try:
            days = int(v)
            if days > 1000:
                return (pd.Timestamp('1900-01-01')+pd.Timedelta(days=days)).date()
            else:
                return None
        except ValueError:
            pass
        
        return None

    else:
        try:
            return v.date()
        except AttributeError:
            return None
        
source['birthday'] = pd.to_datetime(source.birthdate_converted.apply(convt_date))
   
#Mop up wacky dates

source['birthday'] =    source.birthday.where(source.birthday.dt.year > 1990, None) 
    
source['checkin_timestamp'] = pd.to_datetime(source.checkin_timestamp)
source['checkout_timestamp'] = pd.to_datetime(source.checkout_timestamp)
     
cols = 'location first_name last_name sex grade birthday feeling start_date submit_date network_id form clientid tablecloth_link qr_code_link'.split()

for c in cols:
    try:
        source[c]
    except KeyError:
        source[c] = None

source = source.sort_values('checkin_timestamp')\
    .drop_duplicates(subset=['first_name', 'last_name', 'checkin_timestamp', 'checkout_timestamp'])       
        
source.head().T

,0,6647,6648,6649,6650
source_file,sept18,oct17,oct17,oct17,oct17
location,ElCajon,ElCajon,ElCajon,ElCajon,ElCajon
first_name,Aelen,Yemen,Regina,Abdallah,Sarah
last_name,Putrus,Putrus,Alkass,Saleh,Saleh
sex,Girl,Boy,Girl,Boy,Girl
grade,1,4,1,1,3
birthdate,2011-04-13,2008-05-05,2012-08-26,2011-03-23,2009-04-27
feeling,heart eyes,smiling face,heart eyes,screaming in fear,smiling face
checkout_feeling,,,,,
checkin_timestamp,2017-10-25 23:29:23,2017-10-25 23:30:37,2017-10-25 23:32:42,2017-10-25 23:46:01,2017-10-25 23:47:51


In [23]:
source['start_date'] = source.checkin_timestamp

ci = source[source.checkin_timestamp.notnull()].copy()
ci['submit_date'] = ci['checkin_timestamp']
ci['form'] = 'Checkin'

co = source[source.checkout_timestamp.notnull()].copy()
co['submit_date'] = co['checkout_timestamp']
co['form'] = 'Checkout'

new_source = pd.concat([ci, co])[cols].sort_values('submit_date')
print(len(new_source))
new_source


6702


,location,first_name,last_name,sex,grade,birthday,feeling,start_date,submit_date,network_id,form,clientid,tablecloth_link,qr_code_link
0,ElCajon,Aelen,Putrus,Girl,1,2011-04-13,heart eyes,2017-10-25 23:29:23,2017-10-25 23:29:23,None,Checkin,None,None,None
6647,ElCajon,Yemen,Putrus,Boy,4,2008-05-05,smiling face,2017-10-25 23:30:37,2017-10-25 23:30:37,None,Checkin,None,None,None
6648,ElCajon,Regina,Alkass,Girl,1,2012-08-26,heart eyes,2017-10-25 23:32:42,2017-10-25 23:32:42,None,Checkin,None,None,None
6649,ElCajon,Abdallah,Saleh,Boy,1,2011-03-23,screaming in fear,2017-10-25 23:46:01,2017-10-25 23:46:01,None,Checkin,None,None,None
6650,ElCajon,Sarah,Saleh,Girl,3,2009-04-27,smiling face,2017-10-25 23:47:51,2017-10-25 23:47:51,None,Checkin,None,None,None
6651,ElCajon,Mohammad,Saleh,Boy,5,2007-06-13,smiling face,2017-10-25 23:48:50,2017-10-25 23:48:50,None,Checkin,None,None,None
6652,ElCajon,Kamilya,Alkass,Girl,1,2012-04-28,smiling face,2017-10-26 00:11:11,2017-10-26 00:11:11,None,Checkin,None,None,None
6653,ElCajon,Daniel,Putrus,Boy,1,2015-02-21,smiling face,2017-10-26 00:12:56,2017-10-26 00:12:56,None,Checkin,None,None,None
6654,ElCajon,Regina,Alkass,Girl,1,2012-08-26,,NaT,2017-10-26 00:50:17,None,Checkout,None,None,None
6655,ElCajon,Kamilya,Alkass,Girl,1,2012-04-28,,NaT,2017-10-26 00:51:02,None,Checkout,None,None,None
